In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings("ignore")


## 開啟"D:\kc.hsu\OneDrive - Bridgestone\價格相關\預算\2023\RT2\20230320_RT2_assumption.xlsx"
- 將Rolling明細中的規格和每月數量複製出來
- 將23OB的每月數量複製出來 

In [57]:
# 貼上Rolling明細，並轉置成可使用的數據格式
df = pd.read_clipboard()
df = df.set_index(["PCODE", "規格", "花紋", "產地"]).stack().reset_index(name="數量")
df.columns = ["PCODE", "規格", "花紋", "產地", "月份", "數量"]
df

,PCODE,規格,花紋,產地,月份,數量
0,TBR06465,825R20,R210,BSJ,3月,0.0
1,TBR06465,825R20,R210,BSJ,4月,1579.0
2,TBR06465,825R20,R210,BSJ,5月,939.0
3,TBR06465,825R20,R210,BSJ,6月,605.0
4,TBR06465,825R20,R210,BSJ,7月,2463.0
...,...,...,...,...,...,...
1045,LSR03765,235/75R17.5,R294,BSJ,8月,180.0
1046,LSR03765,235/75R17.5,R294,BSJ,9月,250.0
1047,LSR03765,235/75R17.5,R294,BSJ,10月,250.0
1048,LSR03765,235/75R17.5,R294,BSJ,11月,280.0


In [71]:
ob_pat_dict = dict(zip(ob["商品代號"], ob["花紋"]))
df["花紋"] = df["PCODE"].map(ob_pat_dict)

In [81]:
rolling_result = df.groupby(["規格", "花紋", "產地", "月份"])[["數量"]].sum().reset_index()

In [67]:
# 確認OB商品代號都存在於rolling的PCODE裡面
for item in ob["商品代號"].unique():
    if item in df["PCODE"].unique():
        pass
    else:
        print(item)

# 處理OB數據
- 讀取檔案
- 將INCH和SECTION整合為規格加寸別
- 篩選出輪胎行通路
- 將PCODE整理為Rolling目前訂貨的PCODE

In [8]:
# 讀取size_data的23OB數據
ob = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\價格相關\預算\2023\RT2\20230320_RT2_assumption.xlsx", sheet_name="size_data")

In [30]:
ob["寸別"] = ob["INCH"].astype(str).map(lambda x: "R" + x[0:-1] + "." + x[-1] if len(x) > 2 else "R" + x)

In [105]:
ob["SECTION"] = ob["SECTION"].astype(str).map(lambda x: "9.5" if x == "95" else x)

In [106]:
ob["size"] = ob["SECTION"] + ob["寸別"]

In [82]:
ob = ob[(ob["大通路分類"] == "REP") & (ob["營業所"] != "本社") & (ob["通路明細"].isin(['A', 'FMC', '一般店', 'CD', '中盤', 'DS',
       'TK', 'MM', 'FMC BFP', 'TTC', 'BFP', '量販店', 'OT', '加油站', '輪胎行'])) & (ob["預算/前實績"] == "23OB") & (~ob["月份"].isin(["Jan", "Feb"]))]

In [61]:
trans_dict = {"LSR08406": "LSR03459", "TBR0J600": "TBR0J795", "TBR05258": "TBR00328", "TBR0J394": "TBR0J660", "TBR0J458": "TBR0J658", "TBR0J52F": "TBR0JA4F"}

In [64]:
ob["商品代號"] = ob["商品代號"].map(lambda x: trans_dict[x] if x in trans_dict.keys() else x)

In [77]:
ob["產地"] = ob["商品代號"].map(lambda x: "BTMT" if x.endswith("F") else "BTMT" if "0J" in x else "TBSC" if "0L" in x else "BSCN" if "0H" in x else "BSJ")

In [90]:
en_mon = ['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
cn_mon = ['3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']
mon_dict = dict(zip(en_mon, cn_mon))
ob["月份"] = ob["月份"].map(mon_dict)

In [108]:
ob_result = ob.groupby(["size", "花紋", "產地", "月份"])[["條數"]].sum().reset_index()

In [109]:
ob_result.columns = ["規格", "花紋", "產地", "月份", "OB條數"]

In [110]:
rolling_result.columns = ["規格", "花紋", "產地", "月份", "rolling條數"]

In [119]:
final_result = pd.merge(ob_result, rolling_result, on=["規格", "花紋", "產地", "月份"], how="left").fillna(0)

In [123]:
final_result.to_clipboard()

In [121]:
ob_result["OB條數"].sum()

76087

In [122]:
rolling_result["rolling條數"].sum()

99043.5